In [42]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown, VBox, HBox, Button, interactive, Output
from IPython.display import display

In [43]:
img_file_path = '/home/sysadmin/MS-2D-IMG/datasets/IBD_2D/top_512_patch_224x224_overlap_0x0_bin_size_0.01/ST000923-C8-pos/CD/top_512_patch_224x224_overlap_0x0_mz_200.0-1100.0_bin_size_0.01_0024_XAV_iHMP2_LIP_SM-6CAJC_CD.npz'

img = np.load(img_file_path)['patches']
print(f"数据形状: {img.shape}")

数据形状: (512, 224, 224)


In [45]:
def create_interactive_viewer():
    cmaps = ['viridis', 'plasma', 'inferno', 'magma', 'cividis', 
             'gray', 'hot', 'cool', 'spring', 'summer', 'autumn', 'winter',
             'bone', 'copper', 'RdYlBu', 'Spectral']

    # 控件定义
    channel_slider = IntSlider(min=0, max=img.shape[0]-1, value=0, description='Channel:')
    vmin_slider = FloatSlider(min=float(img.min()), max=float(img.max()), 
                             value=np.percentile(img, 1), description='Min:')
    vmax_slider = FloatSlider(min=float(img.min()), max=float(img.max()),
                             value=np.percentile(img, 99), description='Max:')
    cmap_dropdown = Dropdown(options=cmaps, value='gray', description='Colormap:')
    zoom_slider = FloatSlider(min=0.1, max=2.0, value=1.0, description='Zoom:')
    output = Output()

    # 自动范围调整函数
    def auto_range(_=None):
        current_channel = channel_slider.value
        vmin_slider.value = np.percentile(img[current_channel], 1)
        vmax_slider.value = np.percentile(img[current_channel], 99)

    # 事件绑定
    channel_slider.observe(auto_range, names='value')
    auto_range_btn = Button(description="Auto Range")
    auto_range_btn.on_click(auto_range)

    # 绘图函数
    def update_plot(**kwargs):
        with output:
            output.clear_output(wait=True)  # 清除旧输出
            plt.close('all')  # 关闭所有旧图形

            fig = plt.figure(figsize=(8*kwargs['zoom'], 8*kwargs['zoom']))
            plt.imshow(img[kwargs['channel']], 
                      cmap=kwargs['cmap'],
                      vmin=kwargs['vmin'],
                      vmax=kwargs['vmax'])
            plt.colorbar()
            plt.title(f'Channel {kwargs["channel"]}\nIntensity: {kwargs["vmin"]:.2f}-{kwargs["vmax"]:.2f}')
            plt.axis('off')
            plt.show()

    # 创建交互对象
    controls = VBox([
        HBox([channel_slider, auto_range_btn]),
        HBox([vmin_slider, vmax_slider]),
        HBox([cmap_dropdown, zoom_slider])
    ])

    # 初始化
    auto_range()
    interactive_plot = interactive(update_plot, 
                                  {'manual': False},
                                  channel=channel_slider,
                                  vmin=vmin_slider,
                                  vmax=vmax_slider,
                                  cmap=cmap_dropdown,
                                  zoom=zoom_slider)

    # 显示界面
    display(VBox([controls, output]))

In [47]:
create_interactive_viewer()